# Project - Parameters with Highest Impact on House Price Class

![Data Science Workflow](img/ds-workflow.png)

## Goal of Project
- The real estate dealer from last assignment calls back and clarifies his objective
- Not so interested in finding what matters most to find house price, but more in which range a house is in.
- There are 3 classes: 33% cheapest, 33% mid-range, 33% expensive houses.
- He needs to find which 10 parameters matters most to determine that

## Step 1: Acquire
- Explore problem
- Identify data
- Import data

### Step 1.a: Import libraries
- Execute the cell below (SHIFT + ENTER)
- NOTE: You might need to install mlxtend, if so, run the following in a cell
```
!pip install mlxtend
```

In [15]:
import pandas as pd
from sklearn.feature_selection import VarianceThreshold
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from mlxtend.feature_selection import SequentialFeatureSelector as SFS

### Step 1.b: Read the data
- Use ```pd.read_parquet()``` to read the file `files/house_sales.parquet`
- NOTE: Remember to assign the result to a variable (e.g., ```data```)
- Apply ```.head()``` on the data to see all is as expected

In [2]:
data = pd.read_parquet('files/house_sales.parquet')
data.head()

,MSSubClass,LotFrontage,LotArea,Street,LotShape,LandContour,Utilities,LandSlope,OverallQual,OverallCond,...,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,PoolQC,MiscVal,MoSold,YrSold,SalePrice
Id,,,,,,,,,,,,,,,,,,,,,
1,60,65.0,8450,1,3,3,3,2,7,5,...,61,0,0,0,0,NaN,0,2,2008,208500
2,20,80.0,9600,1,3,3,3,2,6,8,...,0,0,0,0,0,NaN,0,5,2007,181500
3,60,68.0,11250,1,2,3,3,2,7,5,...,42,0,0,0,0,NaN,0,9,2008,223500
4,70,60.0,9550,1,2,3,3,2,7,5,...,35,272,0,0,0,NaN,0,2,2006,140000
5,60,84.0,14260,1,2,3,3,2,8,5,...,84,0,0,0,0,NaN,0,12,2008,250000


### Step 1.c: Inspect the data
- Check the number of rows and columns
    - HINT: `.shape`

In [3]:
data.shape

(1460, 56)

## Step 2: Prepare
- Explore data
- Visualize ideas
- Cleaning data

### Step 2.a: Check the data types
- This step tells you if some numeric column is not represented numeric.
- Get the data types by ```.dtypes```

In [4]:
data.dtypes

MSSubClass         int64
LotFrontage      float64
LotArea            int64
Street             int64
LotShape           int64
LandContour        int64
Utilities          int64
LandSlope          int64
OverallQual        int64
OverallCond        int64
YearBuilt          int64
YearRemodAdd       int64
MasVnrArea       float64
ExterQual          int64
ExterCond          int64
BsmtQual         float64
BsmtCond         float64
BsmtExposure     float64
BsmtFinType1     float64
BsmtFinSF1         int64
BsmtFinType2     float64
BsmtFinSF2         int64
BsmtUnfSF          int64
TotalBsmtSF        int64
HeatingQC          int64
CentralAir         int64
1stFlrSF           int64
2ndFlrSF           int64
LowQualFinSF       int64
GrLivArea          int64
BsmtFullBath       int64
BsmtHalfBath       int64
FullBath           int64
HalfBath           int64
BedroomAbvGr       int64
KitchenAbvGr       int64
KitchenQual        int64
TotRmsAbvGrd       int64
Fireplaces         int64
FireplaceQu      float64


### Step 2.b: Check for null (missing) values
- Let's check if any features are not valuable
- Use ```.info()```
- Should we remove any?
    - You can remove features (columns):
    ```Python
data.drop([<column_name>, ..., <column_name>], axis=1)
```

In [5]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1460 entries, 1 to 1460
Data columns (total 56 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   MSSubClass     1460 non-null   int64  
 1   LotFrontage    1201 non-null   float64
 2   LotArea        1460 non-null   int64  
 3   Street         1460 non-null   int64  
 4   LotShape       1460 non-null   int64  
 5   LandContour    1460 non-null   int64  
 6   Utilities      1460 non-null   int64  
 7   LandSlope      1460 non-null   int64  
 8   OverallQual    1460 non-null   int64  
 9   OverallCond    1460 non-null   int64  
 10  YearBuilt      1460 non-null   int64  
 11  YearRemodAdd   1460 non-null   int64  
 12  MasVnrArea     1452 non-null   float64
 13  ExterQual      1460 non-null   int64  
 14  ExterCond      1460 non-null   int64  
 15  BsmtQual       1423 non-null   float64
 16  BsmtCond       1423 non-null   float64
 17  BsmtExposure   1422 non-null   float64
 18  BsmtFinT

In [6]:
data = data.drop(['PoolQC'], axis=1)

## Step 3: Analyze
- Feature selection
- Model selection
- Analyze data

### Step 3.a: Quasi constant features
- Let see if there are any quasi features
- Create a `VarianceThreshold(threshold=0.01)` and fit it
- The features that are not quasi constant are given by `sel.get_feature_names_out()`
- Get all the qausi features as with list comprehension

In [9]:
sel = VarianceThreshold(threshold=0.01)
sel.fit(data)

VarianceThreshold(threshold=0.01)

In [10]:
quasi_features = [col for col in data.columns if col not in sel.get_feature_names_out()]

### Step 3.b: Correlated features
- Calculate the correlation matrix `corr_matrix` and inspect it
    - HINT: use `.corr()`
- Get all the correlated features
    - HINT: A feature is correlated to a feature before it if
```Python
(corr_matrix[feature].iloc[:corr_matrix.columns.get_loc(feature)] > 0.8).any()
```
    - HINT: Use list comprehension to get a list of the correlated features

In [11]:
corr_matrix = data.corr()

In [12]:
corr_features = [feature for feature in corr_matrix.columns if (corr_matrix[feature].iloc[:corr_matrix.columns.get_loc(feature)] > 0.8).any()]
corr_features

['BsmtFinSF2', '1stFlrSF', 'TotRmsAbvGrd', 'GarageYrBlt', 'GarageArea']

### Step 3.c: Prepare training and test set
- Create 3 categorical price ranges using `qcut`
    - HINT: `pd.qcut(data['SalePrice'], q=3, labels=[1, 2, 3])`
- Assign all features in `X`
    - HINT: Use `.drop(['SalePrice', 'Target'] + quasi_features + corr_features, axis=1)`
        - (assuming the same naming)
- Assign the target to `y`
    - HINT: The target is column `Target`
- Split into train and test using `train_test_split`

In [18]:
data['Target'] = pd.qcut(data['SalePrice'], q=3, labels=[1,2,3])

X = data.drop(['SalePrice', 'Target'] + quasi_features + corr_features, axis=1)
y = data['Target']

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=0)

### Step 3.d: 10 best features for LinearRegression model
- Use the `SFS` to find 10 best features for a `LinearRegression` model
    - HINT: `SFS(LinearRegression(), k_features=10, verbose=2)`
    - HINT: when fitting fill missing values or remove them
        - Notice: ideally we would investigate them further to find appropriate values
- You get the best feature index from `.k_feature_idx_`

In [19]:
sfs = SFS(KNeighborsClassifier(), k_features=10, verbose=2)

In [20]:
sfs.fit(X_train.fillna(-1), y_train)

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done  47 out of  47 | elapsed:    1.4s finished

[2021-12-11 13:55:44] Features: 1/10 -- score: 0.6374429223744291[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done  46 out of  46 | elapsed:    1.3s finished

[2021-12-11 13:55:45] Features: 2/10 -- score: 0.6949771689497717[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done  45 out of  45 | elapsed:    1.2s finished

[2021-12-11 13:55:46] Features: 3/10 -- score: 0.7223744292237442[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done  

SequentialFeatureSelector(estimator=KNeighborsClassifier(), k_features=10,
                          verbose=2)

### Step 3.e: Explore the features
- Let's try to explore the features
    - HINT: The features can be accessed by `sfs.k_feature_idx_`
    - HINT: Get the feature names by: `X_train.columns[list(sfs.k_feature_idx_)]`
- Try to list them according to correlation score
    - HINT: This is a bit more advanced Python
    ```Python
for item in X_train.columns[list(sfs.k_feature_idx_)]:
    loc = corr_matrix['SalePrice'].sort_values(ascending=False).index.get_loc(item)
    print(item, loc)
    ```
- Does the result surprise you?
- Does it change your recommendations?

In [21]:
sfs.k_feature_idx_

(0, 5, 6, 15, 22, 28, 31, 32, 35, 37)

In [25]:
X_train.columns[list(sfs.k_feature_idx_)]

Index(['MSSubClass', 'LandSlope', 'OverallQual', 'BsmtExposure', 'CentralAir',
       'FullBath', 'KitchenAbvGr', 'KitchenQual', 'GarageCars', 'GarageCond'],
      dtype='object')

In [32]:
for item in X_train.columns[list(sfs.k_feature_idx_)]:
    loc = corr_matrix['SalePrice'].sort_values(ascending=False).index.get_loc(item)
    print(item, loc)

MSSubClass 51
LandSlope 49
OverallQual 1
BsmtExposure 19
CentralAir 28
FullBath 10
KitchenAbvGr 53
KitchenQual 4
GarageCars 6
GarageCond 34


## Step 4: Report
- Present findings
- Visualize results
- Credibility counts

### Step 4.a: Present findings
- Use the analysis from Step 3 to figures out how to present your findings
- Try to think how the real estate dealer can use these findings

## Step 5: Actions
- Use insights
- Measure impact
- Main goal

### Step 5.a: Measure impact
- Can we help the dealer to use these insights?